In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from huggingface_hub import login

In [2]:
df1 = pd.read_csv("hf://datasets/FrancophonIA/Fon_French_Daily_Dialogues_Parallel_Data/train.csv")


In [3]:
splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
train2 = pd.read_parquet("hf://datasets/beethogedeon/fr_fon/" + splits["train"])

In [4]:
test2 = pd.read_parquet("hf://datasets/beethogedeon/fr_fon/" + splits["test"])

In [5]:
df1

,Fon_Sentence,French_Translation
0,é ɖò ayìjúnjɔ́n,elle est assis
1,mí ɖò ayìjúnjɔ́n,nous sommes assis
2,e ɖo azɔmɛ,elle est à l'école
3,mí ɖo azɔmɛ,nous sommes à l'école
4,e ɖo xwe gbe,elle est à la maison
...,...,...
25372,fi bi fa amɔ gbɛ ɔ mɛ nyɔ,il fait froid mais le temps est agréable
25373,ali wegɔ ɔ aɖisimε,la deuxième voie à droite
25374,vodún tɛ́ a nɔ sɛ̀n,quel est le vodoun que tu sers
25375,un gbɛ̆,non


In [6]:
train2

,french,fon
0,Évitez de ne pas laisser le tissu surchauffer ...,Ma lɔn nú avɔ ɔ ni xú zɛ xwé wu ó (enɛ sixu zɔ...
1,Le centre de la méditation tibétaine est le yo...,Nǔ e jí è nɔ lin tamɛ dó ɖò Tibet é wɛ nyí nǔm...
2,"Madagascar est de loin le plus grand pays, et ...","Madagascar wɛ nyí tò ɖaxó hugǎn ɔ, bo tlɛ nyí ..."
3,"Comme exemple, les citoyens américains au Moye...","Ði kpɔ́ndéwú ɔ, toví Amɛlika tɔn e ɖò Moyen-Or..."
4,Les scientifiques du Centre national des ourag...,Nǔnywɛ xwitixwitizɔ́watɔ́ e ɖò Centre National...
...,...,...
3321,"Les accusés s'appellent Baba Kanjar, Bhutha Ka...","Mɛ e è dóhwɛ lɛ é wɛ nyí Baba Kanjar, Bhutha K..."
3322,Alors qu'ils patientaient sous une chaleur de ...,Hwenu e ye ɖò te kpɔ́n wɛ ɖò zozo 32°C tɔn mɛ ...
3323,Les troupes renforçant les positions avancées ...,Ahwanfuntɔ́ ɖěɖee nɔ hɛn tɛn nukɔntɔn lɛ lidǒ ...
3324,"Les plages sont nombreuses, car Auckland est à...","Xùtó lɛ sukpɔ́, ɖó Auckland gbɔn tɔjihunglintɛ..."


In [7]:
# replace Fon_Sentence with fon and French_Translation with french
df1 = df1.rename(columns={"Fon_Sentence": "fon", "French_Translation": "french"})

In [8]:
df1

,fon,french
0,é ɖò ayìjúnjɔ́n,elle est assis
1,mí ɖò ayìjúnjɔ́n,nous sommes assis
2,e ɖo azɔmɛ,elle est à l'école
3,mí ɖo azɔmɛ,nous sommes à l'école
4,e ɖo xwe gbe,elle est à la maison
...,...,...
25372,fi bi fa amɔ gbɛ ɔ mɛ nyɔ,il fait froid mais le temps est agréable
25373,ali wegɔ ɔ aɖisimε,la deuxième voie à droite
25374,vodún tɛ́ a nɔ sɛ̀n,quel est le vodoun que tu sers
25375,un gbɛ̆,non


In [9]:
# concatenate df1, train2 and test2
df = pd.concat([df1, train2, test2])

In [10]:
df

,fon,french
0,é ɖò ayìjúnjɔ́n,elle est assis
1,mí ɖò ayìjúnjɔ́n,nous sommes assis
2,e ɖo azɔmɛ,elle est à l'école
3,mí ɖo azɔmɛ,nous sommes à l'école
4,e ɖo xwe gbe,elle est à la maison
...,...,...
827,"Ɖo Kundalini Yoga mɛ ɔ, Kundalini hlɔnhlɔn (hl...","Avec le Kundalini Yoga, l’énergie Kundalini (é..."
828,Zayat gán ɖò alitawoví ɔ mɛ bo ma gblewu ǎ.,Zayat est sorti indemne de l'accident.
829,Azɔn enɛ ɔ zɔ́n bɔ acɛkpikpa Inde tɔn bló tuto...,L’épidémie a incité le gouvernement indien à p...
830,"Ðó hwǐ ɔ sín dò nɔ kɔ́n kpɛɖé wutu ɔ, enyi hwǐ...",Comme le bas de la lame est légèrement incurvé...


In [11]:
# remove duplicates
df = df.drop_duplicates(subset=["fon", "french"])
df

,fon,french
0,é ɖò ayìjúnjɔ́n,elle est assis
1,mí ɖò ayìjúnjɔ́n,nous sommes assis
2,e ɖo azɔmɛ,elle est à l'école
3,mí ɖo azɔmɛ,nous sommes à l'école
4,e ɖo xwe gbe,elle est à la maison
...,...,...
762,Angel (2006) xlɛ Continuum sín ali ɔ ɖi wlɛnwí...,Angel (2006) présente l'approche Continuum com...
779,"É nyɔ́ wà ɔ, ninɔmɛ nǔ lɛ tɔn ɛnɛ lɛ wɛ (ɖò tu...",Il s'agissait plutôt des quatre états de la ma...
800,Ðò toxo gègě ɖevo lɛ mɛ ɖò Italie kpo gbɛ̀ ɔ b...,Dans de nombreuses autres villes d'Italie et d...
816,"Nǔkplɔnmɛ ɔ xlɛ́ ɖɔ linkpɔ́nzɔn, xɛsi kpo nǔ b...","L'étude a révélé que la dépression, la peur et..."


In [13]:
# remove rows with empty fon or french
df = df.dropna(subset=["fon", "french"])
df

,fon,french
0,é ɖò ayìjúnjɔ́n,elle est assis
1,mí ɖò ayìjúnjɔ́n,nous sommes assis
2,e ɖo azɔmɛ,elle est à l'école
3,mí ɖo azɔmɛ,nous sommes à l'école
4,e ɖo xwe gbe,elle est à la maison
...,...,...
762,Angel (2006) xlɛ Continuum sín ali ɔ ɖi wlɛnwí...,Angel (2006) présente l'approche Continuum com...
779,"É nyɔ́ wà ɔ, ninɔmɛ nǔ lɛ tɔn ɛnɛ lɛ wɛ (ɖò tu...",Il s'agissait plutôt des quatre états de la ma...
800,Ðò toxo gègě ɖevo lɛ mɛ ɖò Italie kpo gbɛ̀ ɔ b...,Dans de nombreuses autres villes d'Italie et d...
816,"Nǔkplɔnmɛ ɔ xlɛ́ ɖɔ linkpɔ́nzɔn, xɛsi kpo nǔ b...","L'étude a révélé que la dépression, la peur et..."


In [15]:
# Split the data into train, test, and validation sets
train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [23]:
#  Delete the index column
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
val = val.reset_index(drop=True)

In [24]:
# Publish the datasets(train, test, val) to the huggingface hub
train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)
val_dataset = Dataset.from_pandas(val)
dataset_dict = DatasetDict({"train": train_dataset, "test": test_dataset, "validation": val_dataset})
dataset_dict.push_to_hub("jonathansuru/fr_fon")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/17 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/6 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]